<a href="https://colab.research.google.com/github/cogilv25/21010093uhi_DataAnalytics/blob/main/DataAnalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Collection
## Crime
### Check for duplicate records
The following query was run to check for reports made with the same case number:
```
SELECT * FROM `bigquery-public-data.chicago_crime.crime`
WHERE `case_number` IN
  (
    SELECT `case_number` FROM `bigquery-public-data.chicago_crime.crime`
    GROUP BY `case_number` HAVING COUNT(*) > 1
  )
ORDER BY `case_number` ASC;
```
This returned 50 records all of which were homicide cases and it was noted that where multiple records shared a case_number there were no relevant differences. Further research uncovered that the database has a record for each victim in the case of homicides and as these are single incidents only a single record needs to be kept for the purpose of predicting incidents.

### Construct query
The query to count the number of crimes reported for each date excluding any dates after 04/05/2025 as 05/05/2025 is incomplete at the time of writing.
```
SELECT CAST(`date` as DATE) as `date`, COUNT(DISTINCT(`case_number`)) as `reports`
FROM `bigquery-public-data.chicago_crime.crime` WHERE `date` < "2025-05-05"
GROUP BY `date`
```

## Weather
### Find a suitable weather station
Google Maps was used in order to find a reasonable set of bounds for central Chicago which could then be used to query National Oceanic and Atmospheric Administration's Global Surface Summary of the Day Database (NOAA GSOD) using BigQuery for a suitable weather station to use. The chosen bounds were between 41.64436 and 42.01836 latitude and between -87.84245 and -87.52453 longitude as shown image??:

We can then run the following query using BigQuery:
```
SELECT * FROM `bigquery-public-data.noaa_gsod.stations` WHERE
lat > 41.64436 AND lat < 42.01836 AND lon > -87.84245 AND lon < -87.52453
ORDER BY `begin` ASC;
```
Ordering by end to get the weather stations with the most up to date data then begin to get the longest running stations since we need data from 2001 to 2025. From the list returned the data coverage is checked and it is found that Chicago Midway Airport has 100% data coverage going back to 1997 so this source is chosen.

### Construct query
The query to collect the weather data needed from each database for each year and combine it together is created (a "date" column is created for easy joining with the crime data):
```
SELECT DATE(CAST(year as INT64),CAST(mo as INT64),CAST(da as INT64)) as `date`, `temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp` FROM
(
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp` FROM `bigquery-public-data.noaa_gsod.gsod2001` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp` FROM `bigquery-public-data.noaa_gsod.gsod2002` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2003` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2004` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2005` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2006` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2007` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2008` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2009` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2010` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2011` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2012` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2013` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2014` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2015` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2016` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2017` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2018` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2019` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2020` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2021` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2022` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2023` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2024` WHERE `stn`="725340"
  UNION ALL
  SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2025` WHERE `stn`="725340"
)
```
## Collate data
The 2 queries need to be joined together by using the date field which will give the full dataset that can be further processed and used in prediction models. Additionally a day field is generated which represents the day of the week as it is possible that this could be a useful metric for prediction. We could create a view for each data set then query those views to collate the data but since the query is still quite small it's easier to do it in a single query where if anything needs changed we don't need to modify multiple files.
```
# Crime Data
SELECT `crime`.`date`, EXTRACT(DAYOFWEEK FROM `crime`.`date`) AS `day`, `reports`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp` FROM
(
  SELECT CAST(`date` as DATE) as `date`, COUNT(DISTINCT(`case_number`)) as `reports`
  FROM `bigquery-public-data.chicago_crime.crime`  WHERE `date` < "2025-05-05"
  GROUP BY `date`
) AS `crime`

# Left join to remove any weather records we don't have crime records for.
LEFT JOIN

# Weather Data
(
  SELECT DATE(CAST(year as INT64),CAST(mo as INT64),CAST(da as INT64)) as `date`, `temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp` FROM
  (
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp` FROM `bigquery-public-data.noaa_gsod.gsod2001` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp` FROM `bigquery-public-data.noaa_gsod.gsod2002` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2003` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2004` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2005` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2006` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2007` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2008` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2009` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2010` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2011` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2012` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2013` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2014` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2015` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2016` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2017` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2018` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2019` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2020` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2021` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2022` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2023` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2024` WHERE `stn`="725340"
    UNION ALL
    SELECT `da`, `mo`, `year`,`temp`, `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`  FROM `bigquery-public-data.noaa_gsod.gsod2025` WHERE `stn`="725340"
  )
) AS `weather`

ON `weather`.`date`=`crime`.`date` ORDER BY `crime`.`date`
```

ERROR: Error in parse(text = input): <text>:1:15: unexpected symbol
1: SELECT * FROM a
                  ^
